In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Importation des données

In [ ]:
data_path = "../data/welddb.data"
column_names = [
    'C', 'Si', 'Mn', 'S', 'P', 'Ni', 'Cr', 'Mo', 'V', 'Cu', 'Co', 'W', 
    'O', 'Ti', 'N', 'Al', 'B', 'Nb', 'Sn', 'As', 'Sb', 
    'Current', 'Voltage', 'AC_DC', 'Electrode_polarity', 'Heat_input', 'Interpass_temp',  
    'Weld_type', 'PWHT_temp', 'PWHT_time', 
    'Yield_strength', 'UTS', 'Elongation', 'Reduction_area',  
    'Charpy_temp', 'Charpy_impact', 'Hardness', 'FATT_50', 
    'Primary_ferrite', 'Ferrite_2nd_phase', 'Acicular_ferrite', 'Martensite', 'Ferrite_carbide', 
    'Weld_ID' 
]
df = pd.read_csv(data_path, sep='\s+', names=column_names, na_values='N') # as the NaN values are represented by 'N' in the data file
df

In [ ]:
categorical_cols = ['AC_DC', 'Electrode_polarity', 'Weld_type', 'Weld_ID']
chemical_cols = ['C','Si','Mn','S','P','Ni','Cr','Mo','V','Cu','Co',
                 'W','O','Ti','N','Al','B','Nb','Sn','As','Sb']

# Data statistics

In [ ]:
df.info()

On remarque que beaucoup de colonnes ont une majorité de valeurs manquantes. 

In [ ]:
print(f"Proportion of missing values: {df.isna().sum().sum() / df.size}")

In [ ]:
df_missing = df.isna().sum().sort_values(ascending=False) / len(df)
plt.figure(figsize=(12,6))
sns.barplot(x=df_missing.index, y=df_missing.values)
plt.xticks(rotation=90)
plt.ylabel('Proportion of missing values')

In [ ]:
df_missing_non_chemical = df_missing.drop(chemical_cols)
plt.figure(figsize=(12,6))
sns.barplot(x=df_missing_non_chemical.index, y=df_missing_non_chemical.values)
plt.xticks(rotation=90)
plt.ylabel('Proportion of missing values')

In [ ]:
df_missing_non_chemical * len(df)

In [ ]:
# Visualize missing data pattern
plt.figure(figsize=(12,8))
sns.heatmap(df.isna(), cbar=False, cmap='viridis')
plt.title('Missing Data Pattern')
plt.show()

In [ ]:
# visualize for non-chemical features only
plt.figure(figsize=(12,8))
sns.heatmap(df.drop(columns=chemical_cols).isna(), cbar=False, cmap='viridis')
plt.title('Missing Data Pattern (Non-Chemical Features)')
plt.show()

On a des colonnes qui ont l'air d'être toujours présentes ensemble et d'autres alors que les autres sont absentes. Comme si l'on avait fusionner des datasets différents sur des colonnes en commun. 

Les colonnes communes seraient les colonnes des éléments chimiques, et les colonnes : ["Current", "Voltage", "AC_DC", "Electrode_polarity", "Heat_input", "Interpass_temp", "Weld_type", "PWHT_temp", "PWHT_time"].

Les colonnes du premier dataset : ["Yield_strength", "UTS", "Elongation", "Reduction_area"]

Deuxième : ["Charpy_temp", "Charpy_impact"]

Troisième : ["Primary_ferrite", "Ferrite_2nd_phase", "Acicular_ferrite", "Martensite", "Ferrite_carbide"]

Hardness et FATT_50 ont l'air seuls. 

In [ ]:
common_columns = chemical_cols + ["Current", "Voltage", "AC_DC", "Electrode_polarity", "Heat_input", "Interpass_temp", "Weld_type", "PWHT_temp", "PWHT_time"]
first_dataset_cols = ["Yield_strength", "UTS", "Elongation", "Reduction_area"]
second_dataset_cols = ["Charpy_temp", "Charpy_impact"]
third_dataset_cols = ["Primary_ferrite", "Ferrite_2nd_phase", "Acicular_ferrite", "Martensite", "Ferrite_carbide"]

df_first = df[common_columns + first_dataset_cols]
df_second = df[common_columns + second_dataset_cols]
df_third = df[common_columns + third_dataset_cols]

In [ ]:
idx_1 = set(list(df_first.dropna(subset=first_dataset_cols, how='all').index))
idx_2 = set(list(df_second.dropna(subset=second_dataset_cols, how='all').index))
idx_3 = set(list(df_third.dropna(subset=third_dataset_cols, how='all').index))

print(f"intersection of all three datasets: {idx_1 & idx_2 & idx_3}")
print(f"intersection of first and second datasets: {idx_1 & idx_2}")
print(f"intersection of first and third datasets: {idx_1 & idx_3}")
print(f"intersection of second and third datasets: {idx_2 & idx_3}")

In [ ]:
print(f"Number of samples in intersection of first and second datasets: {len(idx_1 & idx_2)}")
print(f"Number of samples in intersection of first and third datasets: {len(idx_1 & idx_3)}")
print(f"Number of samples in intersection of second and third datasets: {len(idx_2 & idx_3)}")
print("\n"+"_"*90+"\n")
print(f"Part de valeurs du dataset 2 présentes dans le dataset 1: {len(idx_1 & idx_2)/len(idx_1):.2%}")
print(f"Part de valeurs du dataset 1 présentes dans le dataset 2: {len(idx_1 & idx_2)/len(idx_2):.2%}")
print(f"Part de valeurs du dataset 3 présentes dans le dataset 1: {len(idx_1 & idx_3)/len(idx_1):.2%}")
print(f"Part de valeurs du dataset 1 présentes dans le dataset 3: {len(idx_1 & idx_3)/len(idx_3):.2%}")
print(f"Part de valeurs du dataset 3 présentes dans le dataset 2: {len(idx_2 & idx_3)/len(idx_2):.2%}")
print(f"Part de valeurs du dataset 2 présentes dans le dataset 3: {len(idx_2 & idx_3)/len(idx_3):.2%}")

In [ ]:
len(idx_1), len(idx_2), len(idx_3)

Le dataset 1 et 3 sont complètements indépendants. Le dataset 2 n'a presque pas de valeurs en commun avec le dataset 3, on les considère indépendant aussi. Le dataset 3 n'a que très peu de valeur (98) et est à moitié composé du dataset 2. Pour essayer de faire de l'imputing on a trop peu de valeurs, on drop ces colonnes. 

Le dataset 1 et 2 ont respectivement 17.73% et 16.38% de valeurs communes et composent chacun environ la moitié du dataset. On va pouvoir utiliser ces deux datasets.  

In [ ]:
(df['Hardness'].isna() == False).sum(), (df['FATT_50'].isna() == False).sum()

Hardness et FATT_50 ont toutes 2 trop peu de valeurs, on va les drop. 

# Gestion des colonnes communes 

In [ ]:
df_common = df[common_columns]
df_common

In [ ]:
non_numeric_cols = []
for dtype in df_common.dtypes.unique():
    if dtype == 'object':
        non_numeric_cols.extend(df_common.columns[df_common.dtypes == dtype].tolist())
non_numeric_cols


## Gestion des colonnes chimiques non-numériques

In [ ]:
non_num_chemical_cols = ['S','Mo','V','Cu','Co','W','Ti','N','Al','B','Nb','Sn','As','Sb']

# Lorsque nan on remplace par 0 pour les colonnes chimiques 
df_common.fillna({col: 0 for col in chemical_cols}, inplace=True)

In [ ]:
# On regarde les valeurs uniques pour chaque colonne chimique non-numérique

non_num_vals = {}
for col in non_num_chemical_cols:
    unique_val_col = df_common[col].dropna().unique()
    for val in unique_val_col:
        try:
            float(val)
        except:
            non_num_vals[col] = non_num_vals.get(col, []) + [val]
non_num_vals

In [ ]:
for col in non_num_chemical_cols:
    tmp = df[col].astype(float, errors='ignore')
    tmp.replace(non_num_vals.get(col, []), np.nan, inplace=True)
    tmp.dropna(inplace=True)
    tmp = tmp.astype(float)
    tmp = tmp.describe()
    print(f"{col}:\n{tmp}\n")

Les unités sont différentes entre les colonnes. On va normaliser après avoir remplacer les valeurs. Pour la colonne 'N' on peut prendre le premier nombre qui a l'air d'être le total (tot). 

In [ ]:
df_common.loc[:, 'N'] = df_common['N'].apply(lambda x: float(str(x).split('tot')[0]) if 'tot' in str(x) else x).astype(float)
df_common["N"] = df_common["N"].astype(float)
df_common["N"].describe()

Par simplicité, on va remplacer les valeurs du type "< x" par r*x avec r un nombre aléatoire entre 0.5 et 1.

In [ ]:
for col in non_num_chemical_cols:
    tmp = df[col].astype(float, errors='ignore')
    tmp.replace(non_num_vals.get(col, []), np.nan, inplace=True)
    tmp.dropna(inplace=True)
    tmp = tmp.astype(float)
    
    std = tmp.std()
    df_common.loc[:, col] = df_common[col].apply(lambda x: float(x.split('<')[1]) * np.random.uniform(0.5,1) if '<' in str(x) else x).astype(float)
    df_common[col] = df_common[col].astype(float)

df_common[non_num_chemical_cols].describe()
    

## Gestion des colonnes non chimiques catégorielles.

In [ ]:
no_chem_common_categorical_col = [ 'AC_DC','Electrode_polarity', 'Interpass_temp','Weld_type']

df_common[no_chem_common_categorical_col].info()

Interpass_temp et Weld_type n'ont pas de nan. On commence par gérer celles-ci.

### Gestion des colonnes Interpass_temp et Weld_type

In [ ]:
df_common["Weld_type"].unique() # Colonne purement catégorielle, on peut faire du one-hot encoding

In [ ]:
df_common = pd.get_dummies(df_common, columns=['Weld_type'], prefix='Weld_type', drop_first=False, dtype=int)
df_common

In [ ]:
df_common["Interpass_temp"].unique() # On a que des valeurs numériques ou des intervalles du type x0-x1. 
# On va remplacer les intervalles par leur moyenne. Un nombre aléatoire entre les deux bornes centré sur la moyenne.

In [ ]:
def replace_interval(val):
    if '-' in str(val):
        bounds = val.split('-')
        lower_bound = float(bounds[0])
        upper_bound = float(bounds[1])
        to_ret = np.random.normal(loc=(lower_bound + upper_bound) / 2, scale=(upper_bound - lower_bound) / 7)
        if to_ret < lower_bound:
            to_ret = lower_bound
        if to_ret > upper_bound:
            to_ret = upper_bound
        return to_ret
    else:
        return float(val)
    
test =[replace_interval("100-200") for i in range(100000)]
plt.hist(test, bins=100)
plt.title("Distribution des valeurs remplaçant l'intervalle 100-200 (exemple)")
plt.show()

In [ ]:
df_common.loc[:, "Interpass_temp"] = df_common["Interpass_temp"].apply(replace_interval).astype(float)
df_common["Interpass_temp"] = df_common["Interpass_temp"].astype(float)
df_common["Interpass_temp"].describe()

### Gestion des colonnes AC_DC et Electrode_polarity 

In [ ]:
df_common['AC_DC'].unique(), df_common['Electrode_polarity'].unique()

In [ ]:
sns.heatmap(df_common[['AC_DC', 'Electrode_polarity']].isna())

In [ ]:
df_common[['AC_DC', 'Electrode_polarity']].describe()

Les colonnes sont quasiment constantes : DC apparaît 97% du temps et + apparaît 96% du temps. 

On va tester un imputing avec des modèles robustes au problème de déséquilibre de classe (Arbres).

#### Imputing AC_DC 

In [ ]:
# import Kfold, RandomForestClassifier, XGBoostClassifier, AdaBoostClassifier

from sklearn.model_selection import KFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix

# Pas besoin de scaler les données pour les modèles d'arbres

full = df_common[list(set(df_common.columns) - set(['Electrode_polarity']))].copy()
full.dropna(inplace=True)

cols = list(set(full.columns) - set(['AC_DC', 'Electrode_polarity', 'Current', 'Voltage'])) # On enlève Current et Voltage car nan

X = full[cols]
y = full['AC_DC']

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
model = AdaBoostClassifier(n_estimators=100, random_state=42, algorithm='SAMME')
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    print((model.predict(X_test) == y_test).sum(), "predictions correctes sur", len(y_test))
    print(f"Fold accuracy: {score:.2%}")
    # matric confusionn
    print(confusion_matrix(y_test, model.predict(X_test)))

On a de très bons résultats avec le AdaBoostClassifier, on peut facilement remplacer les valeurs manquantes. 

Résultat peut être trop bon --> voir s'il n'y a pas une merde mais ça a l'air ok

In [ ]:
# Imputing AC_DC
model.fit(X, y)
df_common.loc[df_common['AC_DC'].isna(), 'AC_DC'] = model.predict(df_common.loc[df_common['AC_DC'].isna(), cols])

#### Imputing Electrode_polarity

In [ ]:
full = df_common[list(set(df_common.columns) - set(['AC_DC']))].copy()
full.dropna(inplace=True)

cols = list(set(full.columns) - set(['AC_DC', 'Electrode_polarity', 'Current', 'Voltage'])) # On enlève Current et Voltage car nan

X = full[cols]
y = full['Electrode_polarity']

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
model = AdaBoostClassifier(n_estimators=100, random_state=42, algorithm='SAMME')
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    print((model.predict(X_test) == y_test).sum(), "predictions correctes sur", len(y_test))
    print(f"Fold accuracy: {score:.2%}")
    # matric confusionn
    print(confusion_matrix(y_test, model.predict(X_test)))

Ici aussi les résultats ont l'air très bon. Les modèles n'ignorent pas une des classe. On peut l'utiliser.

In [ ]:
# Imputing Electrode_polarity
model.fit(X, y)
df_common.loc[df_common['Electrode_polarity'].isna(), 'Electrode_polarity'] = model.predict(df_common.loc[df_common['Electrode_polarity'].isna(), cols])

In [ ]:
# AC_DC est catégorielle, on fait du one-hot encoding
df_common = pd.get_dummies(df_common, columns=['AC_DC'], prefix='AC_DC', drop_first=False, dtype=int)

# Electrode_polarity est catégorielle, on fait du one-hot encoding
df_common = pd.get_dummies(df_common, columns=['Electrode_polarity'], prefix='Electrode_polarity', drop_first=False, dtype=int)

#### Gestion des nan dans les colonnes numériques

In [ ]:
df_common.isna().sum()

In [ ]:
cols_na = ['Current', 'Voltage', 'PWHT_temp', 'PWHT_time']

df_common[cols_na].describe()

In [ ]:
df_common[cols_na]

In [ ]:
# Plot of the distributions of the columns with missing values
for col in cols_na:
    plt.figure(figsize=(8,4))
    sns.histplot(df_common[col].dropna(), bins=50, kde=True)
    plt.title(f'Distribution of {col} (non-missing values)')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.show()

Les distributions sont très disparates. On va essayer de remplacer les valeurs manquantes avec les mêmes méthodes que précédemment.

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

models = {}

for col in cols_na:
    full = df_common[list(set(df_common.columns) - (set(cols_na) - {col}))].copy()
    full.dropna(inplace=True)

    cols = list(set(full.columns) - set(cols_na)) 

    X = full[cols]
    y = full[col]

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    model = AdaBoostRegressor(n_estimators=100, random_state=42)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        model.fit(X_train, y_train)
        #score = model.score(X_test, y_test)
        score = np.sqrt(np.mean((model.predict(X_test) - y_test) ** 2))
        print(f"Fold RMSE for {col}: {score}")
        print(f"Fold R2 for {col}: {model.score(X_test, y_test):.2%}\n")
    
    print("_"*40+"\n")
    model.fit(X, y)
    models[col] = (model, cols)

Pour les variables Current et Voltage, le modèle est performant donc on va pouvoir compléter les valeurs manquantes. Pour les colonnes PWHT_temp et PWHT_time les résultats sont bien moins bon et les résultats varient beaucoup d'un plie à l'autre, montrant une grande variance dans les résultats. Etant donné qu'il n'y a que peu de valeurs manquantes (13) on va tout de même les compléter avec le modèle mais ces variables pourraient ne pas être utilisées à l'avenir. 

In [ ]:
# Imputing des colonnes avec les modèles entrainés
for col in cols_na:
    df_common.loc[df_common[col].isna(), col] = models[col][0].predict(df_common.loc[df_common[col].isna(), models[col][1]])

In [ ]:
df_common.info()

On a maintenant uniquement des valeurs numériques et plus aucun nan dans les colonnes communes au 2 datasets. On va pouvoir commencer à faire l'analyse des colonnes spécifiques. 

# Gestion du dataset 1

In [ ]:
import os

dataset_1_plot_path = "../outputs/dataset_1/"
"""Chemin disque pour la sauvegarde des graphiques liés au dataset 1"""
df_1_targets: list[str] = first_dataset_cols
"""Propriétés cibles que l'on veut compléter dans cette analyse"""
df_1_properties: list[str]  = df_common.columns.to_list()

os.makedirs(dataset_1_plot_path, exist_ok=True)

first_dataset_cols # colonnes spécifiques au premier dataset

In [ ]:
df_1_properties

In [ ]:
# Création du dataset avec df_common pour récupérer toutes les valeurs des éléments chimiques
df_1 = pd.concat([df_common, df_first[first_dataset_cols]], axis=1)
"""Dataset complet contenant les colonnes 'df_1_targets' et 'df_1_properties'"""
df_1

In [ ]:
# Vérification de la cohérence de la taille du dataset concaténé
print(f"Taille du dataset 1         : {df_first.shape}")
print(f"Taille du dataset commun    : {df_common.shape}")
print(f"Taille du dataset 1 complet : {df_1.shape}")
assert df_1.shape[0] == df_first.shape[0]
# Si l'assert bloque, alors la concaténation à rajouter des lignes, ce qui n'est pas souhaitable : on veut les colonnes

# Recherche de corrélation (propriétés physiques)

In [ ]:
# Analyse de corrélations potientielles entre les propriétés mécaniques
targets_corr = df_1[df_1_targets].corr()

# Affichage de matrices de corrélation entre les différentes propriétés mécaniques
plt.figure(figsize=(8, 6))
sns.heatmap(targets_corr,
            annot=True,
            fmt='.2f',
            cmap='coolwarm',
            center=0,
            linewidths=0.5,
            cbar_kws={"shrink": 0.8})
plt.title('Corrélation: Propriétés mécaniques')
plt.xlabel('Propriétés mécaniques')
plt.ylabel('Propriétés mécaniques')
plt.tight_layout()
plt.show()

On constate de fortes corrélation entre les propriétés mécaniques (deux à deux) :
- UTS' et 'Yield_strength
- Elongation et Reduction_area

Cela correspond à de la résistance et de la flexibilité.
On peut en apprendre plus en graphant leurs distributions

In [ ]:
plt.figure(figsize=(6,4))
sns.histplot(df_1[df_1_targets].dropna(), bins=50, kde=True)
plt.title(f'Répartition des propriétés mécaniques (non-na)')
plt.xlabel(col)
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.histplot(df_1[['Elongation', 'Reduction_area']].dropna(), bins=50, kde=True)
plt.title(f'Répartition des propriétés mécaniques (non-na)')
plt.xlabel(col)
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.histplot(df_1[['UTS', 'Yield_strength']].dropna(), bins=50, kde=True)
plt.title(f'Répartition des propriétés mécaniques (non-na)')
plt.xlabel(col)
plt.ylabel('Frequency')
plt.show()

Quand on rapproche les distributions des propriétés physiques deux à deux, on se rend compte qu'il existe une nette ressemblance, même au niveau de l'échelle. Elles sont deux à deux des candidates idéales pour l'imputation, à condition qu'elles ne soient pas manquantes en même temps.

In [ ]:
uts_missing = df_1['UTS'].isna()
yield_missing = df_1['Yield_strength'].isna()

cases = {
    'Both Missing'      : (uts_missing & yield_missing).sum(),
    'Only UTS Missing'  : (uts_missing & ~yield_missing).sum(),
    'Only Yield Missing': (~uts_missing & yield_missing).sum(),
    'Both Present'      : (~uts_missing & ~yield_missing).sum()
}

plt.figure(figsize=(10, 5))
plt.bar(cases.keys(), cases.values(), edgecolor='black', linewidth=1.2)
plt.ylabel('Quantité', fontweight='bold')
plt.title('Comparaison des valeurs manquantes', fontweight='bold', fontsize=13)
plt.xticks(rotation=15)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
Elongation_missing = df_1['Elongation'].isna()
Reduction_area_missing = df_1['Reduction_area'].isna()

cases = {
    'Both Missing'               : (Elongation_missing & Reduction_area_missing).sum(),
    'Only Elongation Missing'    : (Elongation_missing & ~Reduction_area_missing).sum(),
    'Only Reduction_area Missing': (~Elongation_missing & Reduction_area_missing).sum(),
    'Both Present'               : (~Elongation_missing & ~Reduction_area_missing).sum()
}

plt.figure(figsize=(10, 5))
plt.bar(cases.keys(), cases.values(), edgecolor='black', linewidth=1.2)
plt.ylabel('Quantité', fontweight='bold')
plt.title('Comparaison des valeurs manquantes', fontweight='bold', fontsize=13)
plt.xticks(rotation=15)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()


Malheureusement, la majorité des cas manquants sont présents sur les deux valeurs à la fois, limitant la possibilité d'utiliser notre plus forte corrélation pour cette tâche.

Seconde vue de la corrélation des valeures manquantes pour les propriétés physiques (sous forme de heatmap)

In [ ]:
missing_correlation = df_1[df_1_targets].isnull().corr()

plt.figure(figsize=(10, 8))
sns.heatmap(missing_correlation, annot=True, cmap='coolwarm', center=0,
            vmin=-1, vmax=1, square=True)
plt.title('Corrélation entre les valeurs manquantes')
plt.tight_layout()
plt.show()

cols_with_na = df_1.columns[df_1.isnull().any()].tolist()
if len(cols_with_na) > 1:
    for i, col1 in enumerate(cols_with_na):
        for col2 in cols_with_na[i+1:]:
            contingency = pd.crosstab(df_1[col1].isnull(),
                                     df_1[col2].isnull(),
                                     margins=True)

In [ ]:
# Correctif des données pour la colonne 'Interpass_temp'
def replace_interval_mean(val):
    if '-' in str(val):
        bounds = val.split('-')
        lower_bound = float(bounds[0])
        upper_bound = float(bounds[1])
        return (lower_bound + upper_bound) / 2
    else:
        return float(val)

# df_1.loc[:, 'Interpass_temp'] = df_1['Interpass_temp'].apply(replace_interval_mean).astype(float)

# Recherche de nouvelles corrélations

In [ ]:
df_1_targets_to_properties_correlation_matrix = df_1[df_1_targets + df_1_properties].corr()
"""La matrice de corrélation entre les targets et properties du dataset 1"""
top_n = 30

In [ ]:
def get_top_correlations(corr_matrix, target_cols, property_cols, top_n=30):
    """
    Extrait les top N corrélations entre targets et properties

    """
    # Assurer que target_cols est une liste
    if isinstance(target_cols, str):
        target_cols = [target_cols]

    corr_targets_properties = corr_matrix.loc[target_cols, property_cols]
    corr_flat = corr_targets_properties.stack()
    top_corr = corr_flat.abs().sort_values(ascending=False).head(top_n)
    top_corr_values = corr_flat[top_corr.index]

    return top_corr_values

In [ ]:
def plot_correlation_barh(corr_values, top_n, title="Corrélations entre Targets vs Properties"):
    """
        Plot un graphique horizontal des corrélations
    """
    plt.figure(figsize=(10, 6))
    colors = ['red' if x < 0 else 'steelblue' for x in corr_values]
    corr_values.plot(kind='barh', color=colors)
    plt.title(f"Top {top_n} {title}")
    plt.xlabel('Correlation')
    plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
"""
corr_targets_properties = df_1_targets_to_properties_correlation_matrix.loc[df_1_targets, df_1_properties]

corr_flat = corr_targets_properties.stack()
top_corr = corr_flat.abs().sort_values(ascending=False).head(top_n)

top_corr_values = corr_flat[top_corr.index]

plt.figure(figsize=(10, 6))
colors = ['red' if x < 0 else 'steelblue' for x in top_corr_values]
top_corr_values.plot(kind='barh', color=colors)
plt.title(f"Top {top_n} des corrélations entre Targets vs Properties (dataset 1)")
plt.xlabel('Correlation')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print("Corrélations les plus fortes :")
print(top_corr_values)
"""

In [ ]:
c = 'Yield_strength'
top_corr_values = get_top_correlations(
    df_1_targets_to_properties_correlation_matrix,
    c,
    df_1_properties,
    top_n
)
plot_correlation_barh(top_corr_values, top_n, f"corrélations entre {c} vs Properties (dataset 1)")

print(f"Corrélations les plus fortes pour {c} :")
print(top_corr_values)

In [ ]:
c = 'UTS'
top_corr_values = get_top_correlations(
    df_1_targets_to_properties_correlation_matrix,
    c,
    df_1_properties,
    top_n
)
plot_correlation_barh(top_corr_values, top_n, f"corrélations entre {c} vs Properties (dataset 1)")

print(f"Corrélations les plus fortes pour {c} :")
print(top_corr_values)

In [ ]:
c = 'Elongation'
top_corr_values = get_top_correlations(
    df_1_targets_to_properties_correlation_matrix,
    c,
    df_1_properties,
    top_n
)
plot_correlation_barh(top_corr_values, top_n, f"corrélations entre {c} vs Properties (dataset 1)")

print(f"Corrélations les plus fortes pour {c} :")
print(top_corr_values)

In [ ]:
c = 'Reduction_area'
top_corr_values = get_top_correlations(
    df_1_targets_to_properties_correlation_matrix,
    c,
    df_1_properties,
    top_n
)
plot_correlation_barh(top_corr_values, top_n, f"corrélations entre {c} vs Properties (dataset 1)")

print(f"Corrélations les plus fortes pour {c} :")
print(top_corr_values)

# Tests d'algorithmes de prédictions pour les valeurs mécaniques manquantes du dataset 1 à partir des propriétés de df_common

# Grid search pour les regresseurs - recherche d'un regresseur optimal paramétré
J'ai déjà exécuté la search une fois et retiré des paramètres des grid pour des raisons de performances

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostRegressor
import xgboost as xgb

def optimize_regressor(data, properties, targets, param_grid, model_class, model_name) -> dict:
    """
    Optimise un modèle de régression pour chaque target via GridSearchCV
    :param
    data : pd.DataFrame - Dataset complet avec features et targets
    properties : list -Noms des colonnes features
    targets : list - Noms des colonnes targets
    param_grid : dict - Grille de paramètres pour GridSearchCV
    model_class : class -Classe du modèle (AdaBoostRegressor, XGBRegressor, etc.)
    model_name : str - Nom du modèle pour affichage
    :return
        dict : Meilleurs modèles par target
    """
    best_models = {}

    for target in targets:
        print(f"\nOptimisation {model_name} pour {target}...")

        train_mask = data[target].notna()
        X_train = data.loc[train_mask, properties]
        y_train = data.loc[train_mask, target]

        grid_search = GridSearchCV(
            estimator=model_class(random_state=42),
            param_grid=param_grid,
            cv=5,
            scoring='r2',
            n_jobs=-1,
            verbose=0
        )

        grid_search.fit(X_train, y_train)

        print(f"{target}: R² = {grid_search.best_score_:.3f} => {grid_search.best_params_}")

        best_models[target] = grid_search.best_estimator_

    return best_models

In [ ]:
data_optimized = df_1[df_1_properties + df_1_targets].copy()
"""Le dataset utilisé pour entraîné les regresseurs puis où on imputeras les valeurs manquantes du dataset 1"""

# AdaBoost

In [ ]:
ada_param_grid = {
    'n_estimators': [75, 100, 150, 200],
    'learning_rate': [0.1, 0.2, 0.5, 1.0],
    'loss': ['square', 'exponential']
}

ada_best_models = optimize_regressor(
    data=data_optimized,
    properties=df_1_properties,
    targets=df_1_targets,
    param_grid=ada_param_grid,
    model_class=AdaBoostRegressor,
    model_name="AdaBoost"
)


Optimisation AdaBoost pour Yield_strength...
Yield_strength: R² = 0.413 => {'learning_rate': 1.0, 'loss': 'square', 'n_estimators': 200}

Optimisation AdaBoost pour UTS...
UTS: R² = 0.469 => {'learning_rate': 0.5, 'loss': 'square', 'n_estimators': 150}

Optimisation AdaBoost pour Elongation...
Elongation: R² = 0.296 => {'learning_rate': 0.5, 'loss': 'exponential', 'n_estimators': 100}

Optimisation AdaBoost pour Reduction_area...
Reduction_area: R² = 0.112 => {'learning_rate': 0.2, 'loss': 'exponential', 'n_estimators': 75}

# XGBoost
Attention, le temps d'exécution de la cellule est long (style 20 minutes) ! J'ai réduit manuellement les paramètres pour trouver la solution optimal plus vite

In [ ]:
xgb_param_grid = {
    'n_estimators': [200, 500],
    'max_depth': [3, 4],
    'learning_rate': [0.2],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.7, 0.8],
    'min_child_weight': [1, 3, 5]
}

xgb_best_models = optimize_regressor(
    data=data_optimized,
    properties=df_1_properties,
    targets=df_1_targets,
    param_grid=xgb_param_grid,
    model_class=xgb.XGBRegressor,
    model_name="XGBoost"
)


Optimisation XGBoost pour Yield_strength...
Yield_strength: R² = 0.591 => {'colsample_bytree': 0.7, 'learning_rate': 0.2, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 500, 'subsample': 1.0}

Optimisation XGBoost pour UTS...
UTS: R² = 0.566 => {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 500, 'subsample': 0.9}

Optimisation XGBoost pour Elongation...
Elongation: R² = 0.338 => {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 4, 'min_child_weight': 5, 'n_estimators': 200, 'subsample': 1.0}

Optimisation XGBoost pour Reduction_area...
Reduction_area: R² = 0.341 => {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 300, 'subsample': 0.8}

In [ ]:
# Imputation par XGBoost (meilleurs résultats)
for target in df_1_targets:
    missing_mask = data_optimized[target].isna()

    if missing_mask.sum() > 0:
        X_missing = data_optimized.loc[missing_mask, df_1_properties]
        predictions = xgb_best_models[target].predict(X_missing)

        data_optimized.loc[missing_mask, target] = predictions

        print(f"{target}: {missing_mask.sum()} valeurs imputées")

print(f"Imputation terminée. Shape: {data_optimized.shape}")
print(f"Valeurs manquantes restantes:\n{data_optimized[df_1_targets].isna().sum()}")

In [ ]:
# @TODO AJOUT DE NOISE A L'IMPUTATION BASE SUR UNE GAUSSIENNE
# @TODO ANALYSE DES RESULTATS DE L'IMPUTATION

# Gestion du dataset 2 

In [ ]:
second_dataset_cols # colonnes spécifiques au second dataset

In [ ]:
df2 = pd.concat([df_common, df_second[second_dataset_cols]], axis=1)
df2

In [ ]:
df2.info()

In [ ]:
print(f"\nValeurs manquantes par colonne:")
print(df2[second_dataset_cols].isna().sum())
print(f"\nProportions manquantes:")
print((df2[second_dataset_cols].isna().sum() / len(df2) * 100).round(2))

# Visualisation des valeurs manquantes
fig, ax = plt.subplots(figsize=(10, 4))
missing_counts = df2[second_dataset_cols].isna().sum()
ax.bar(missing_counts.index, missing_counts.values, color=['red'])
ax.set_ylabel('Nombre de valeurs manquantes')
ax.set_title('Valeurs manquantes dans le dataset 2')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Visualisation du pattern de données manquantes
plt.figure(figsize=(12, 6))
sns.heatmap(df2[second_dataset_cols].isna(), cbar=True, cmap='viridis', yticklabels=False)
plt.title('Pattern de valeurs manquantes - Dataset 2')
plt.tight_layout()
plt.show()

# Statistiques descriptives des colonnes avec données non-manquantes
print(f"\nStatistiques des colonnes du dataset 2:")
print(df2[second_dataset_cols].describe())

On remarque que la température est en Celcius, pour éviter des problèmes liés au signe de la température on transforme ces températures en Kelvin (K) on va le faire à toutes les colonnes de températures
On remarque aussi que quand charpy impact est présent charpy temp aussi ce qui va simplifier notre imputation

In [ ]:
# Identifier toutes les colonnes de température
temp_columns_in_df2 = [col for col in df2.columns if 'temp' in col.lower()]
print(f"\nColonnes de température détectées: {temp_columns_in_df2}")

for temp_col in temp_columns_in_df2:
    if temp_col in df2.columns:
        n_before = df2[temp_col].isna().sum()
        
        # Conversion seulement pour les valeurs non-manquantes
        mask = df2[temp_col].notna()
        df2.loc[mask, temp_col] = df2.loc[mask, temp_col] + 273.15
        
        n_after = df2[temp_col].isna().sum()
        
        print(f"\n{temp_col}:")
        print(f"Valeurs converties: {mask.sum()}")
        print(f"Nouvelles statistiques:")
        print(df2[temp_col].describe())



On va passer à l'imputation toujours par adaboost mais on sauvegarde avant l'imputation pour pouvoir comparer ensuite

In [ ]:
# Sauvegarder les distributions AVANT imputation
distributions_before = {}
for col in second_dataset_cols:
    distributions_before[col] = df2[col].dropna().copy()

In [ ]:
for col in second_dataset_cols:
    if df2[col].notna().sum() > 0:
        plt.figure(figsize=(10, 4))
        sns.histplot(df2[col].dropna(), bins=50, kde=True, color='steelblue')
        plt.title(f'Distribution de {col} (valeurs non-manquantes: {df2[col].notna().sum()})')
        plt.xlabel(col)
        plt.ylabel('Fréquence')
        plt.tight_layout()
        plt.show()

## Imputation des colonnes du dataset 2 avec Adaboost

from sklearn.ensemble import AdaBoostRegressor

df2_full = df2.copy()

imputation_models = {}

for col in second_dataset_cols:
    full = df2_full[list(set(df2_full.columns) - (set(second_dataset_cols) - {col}))].copy()
    full.dropna(inplace=True)

    cols = list(set(full.columns) - set(second_dataset_cols)) 

    X = full[cols]
    y = full[col]

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    model = AdaBoostRegressor(n_estimators=100, random_state=42)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        model.fit(X_train, y_train)
        #score = model.score(X_test, y_test)
        score = np.sqrt(np.mean((model.predict(X_test) - y_test) ** 2))
        print(f"Fold RMSE for {col}: {score}")
        print(f"Fold R2 for {col}: {model.score(X_test, y_test):.2%}\n")
    
    print("_"*40+"\n")
    model.fit(X, y)
    models[col] = (model, cols)


# Application de l'imputation pour les valeurs manquantes
for col in second_dataset_cols:
    if col in models:
        n_missing = df2[col].isna().sum()
        if n_missing > 0:
            model, feature_cols = models[col]
            indices_missing = df2.index[df2[col].isna()]
            
            X_missing = df2_full.loc[indices_missing, feature_cols]
            predictions = model.predict(X_missing)
            
            df2.loc[indices_missing, col] = predictions
            
            print(f"{col}: {n_missing} valeurs imputées")
        else:
            print(f"{col}: aucune valeur manquante")
    else:
        print(f"{col}: imputation non disponible (données insuffisantes)")

# Vérification finale
print(f"\nVérification - Valeurs manquantes restantes:")
print(df2[second_dataset_cols].isna().sum())

# Statistiques finales après imputation
print(f"\nStatistiques du dataset 2 après imputation:")
print(df2[second_dataset_cols].describe())

Comparons les données avant et après l'imputation

In [ ]:
# Comparaison AVANT/APRÈS imputation
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

for idx, col in enumerate(second_dataset_cols):
    ax_before = axes[idx, 0]
    ax_before.hist(distributions_before[col], bins=40, alpha=0.7, color='steelblue', edgecolor='black')
    ax_before.set_title(f'{col} - AVANT imputation (n={len(distributions_before[col])})', fontweight='bold')
    ax_before.set_xlabel(col)
    ax_before.set_ylabel('Fréquence')
    ax_before.grid(True, alpha=0.3)
    ax_before.axvline(distributions_before[col].mean(), color='red', linestyle='--', linewidth=2)
    
    ax_after = axes[idx, 1]
    ax_after.hist(df2[col], bins=40, alpha=0.7, color='coral', edgecolor='black')
    n_imputed = len(df2[col]) - len(distributions_before[col])
    ax_after.set_title(f'{col} - APRÈS imputation (n={len(df2[col])}, {n_imputed} imputées)', fontweight='bold')
    ax_after.set_xlabel(col)
    ax_after.set_ylabel('Fréquence')
    ax_after.grid(True, alpha=0.3)
    ax_after.axvline(df2[col].mean(), color='red', linestyle='--', linewidth=2)

plt.tight_layout()
plt.show()

for col in second_dataset_cols:
    print(f"\n{col}:")
    print(f"  AVANT: n={len(distributions_before[col])}, mean={distributions_before[col].mean():.4f}, std={distributions_before[col].std():.4f}")
    print(f"  APRÈS: n={len(df2[col])}, mean={df2[col].mean():.4f}, std={df2[col].std():.4f}")
    
    delta_mean = df2[col].mean() - distributions_before[col].mean()
    delta_std = df2[col].std() - distributions_before[col].std()
    print(f"  Changements: Δmean={delta_mean:+.4f}, Δstd={delta_std:+.4f}")


In [ ]:
from sklearn.impute import KNNImputer

# Tester plusieurs valeurs de k pour KNN
print("KNN Imputation - Test de différentes valeurs de k")
print("="*60)

scaler = StandardScaler()
numeric_cols = df2_full.select_dtypes(include=[np.number]).columns

k_values = [3, 5, 7, 10]
knn_results = {}

for k in k_values:
    df2_knn_temp = df2_full.copy()
    df2_knn_scaled = df2_full.copy()
    df2_knn_scaled[numeric_cols] = scaler.fit_transform(df2_full[numeric_cols])
    
    imputer_knn = KNNImputer(n_neighbors=k)
    df2_knn_scaled[numeric_cols] = imputer_knn.fit_transform(df2_knn_scaled[numeric_cols])
    df2_knn_temp[numeric_cols] = scaler.inverse_transform(df2_knn_scaled[numeric_cols])
    
    knn_results[k] = df2_knn_temp
    print(f"KNN (k={k}) - Valeurs manquantes restantes: {df2_knn_temp[second_dataset_cols].isna().sum().sum()}")

# Visualisation comparative: AVANT, AdaBoost, KNN(k=3,5,7,10)
fig, axes = plt.subplots(2, 6, figsize=(24, 10))

for idx, col in enumerate(second_dataset_cols):
    # AVANT
    axes[idx, 0].hist(distributions_before[col], bins=40, alpha=0.7, color='steelblue', edgecolor='black')
    axes[idx, 0].set_title(f'{col} - AVANT', fontweight='bold', fontsize=10)
    axes[idx, 0].set_xlabel(col, fontsize=8)
    axes[idx, 0].set_ylabel('Fréquence', fontsize=8)
    axes[idx, 0].grid(True, alpha=0.3)
    axes[idx, 0].axvline(distributions_before[col].mean(), color='red', linestyle='--', linewidth=2)
    axes[idx, 0].tick_params(labelsize=8)
    
    # AdaBoost
    axes[idx, 1].hist(df2[col], bins=40, alpha=0.7, color='coral', edgecolor='black')
    axes[idx, 1].set_title(f'{col} - AdaBoost', fontweight='bold', fontsize=10)
    axes[idx, 1].set_xlabel(col, fontsize=8)
    axes[idx, 1].set_ylabel('Fréquence', fontsize=8)
    axes[idx, 1].grid(True, alpha=0.3)
    axes[idx, 1].axvline(df2[col].mean(), color='red', linestyle='--', linewidth=2)
    axes[idx, 1].tick_params(labelsize=8)
    
    # KNN avec différents k
    for k_idx, k in enumerate(k_values):
        axes[idx, 2+k_idx].hist(knn_results[k][col], bins=40, alpha=0.7, color='lightgreen', edgecolor='black')
        axes[idx, 2+k_idx].set_title(f'{col} - KNN(k={k})', fontweight='bold', fontsize=10)
        axes[idx, 2+k_idx].set_xlabel(col, fontsize=8)
        axes[idx, 2+k_idx].set_ylabel('Fréquence', fontsize=8)
        axes[idx, 2+k_idx].grid(True, alpha=0.3)
        axes[idx, 2+k_idx].axvline(knn_results[k][col].mean(), color='red', linestyle='--', linewidth=2)
        axes[idx, 2+k_idx].tick_params(labelsize=8)

plt.tight_layout()
plt.show()

# Tableau comparatif détaillé
print("\nComparaison détaillée:")
for col in second_dataset_cols:
    print(f"\n{col}:")
    print(f"  AVANT:")
    print(f"    mean={distributions_before[col].mean():.4f}, std={distributions_before[col].std():.4f}")
    print(f"  AdaBoost:")
    ab_mean = df2[col].mean()
    ab_std = df2[col].std()
    print(f"    mean={ab_mean:.4f}, std={ab_std:.4f}")
    print(f"    Δmean={ab_mean - distributions_before[col].mean():+.4f}, Δstd={ab_std - distributions_before[col].std():+.4f}")
    
    for k in k_values:
        knn_mean = knn_results[k][col].mean()
        knn_std = knn_results[k][col].std()
        print(f"  KNN (k={k}):")
        print(f"    mean={knn_mean:.4f}, std={knn_std:.4f}")
        print(f"    Δmean={knn_mean - distributions_before[col].mean():+.4f}, Δstd={knn_std - distributions_before[col].std():+.4f}")

# Résumé des différences par rapport à AVANT
print("\n\nRésumé des écarts (Δmean et Δstd):")
print("-"*80)
for col in second_dataset_cols:
    print(f"\n{col}:")
    print(f"  {'Méthode':<15} {'Δmean':<15} {'Δstd':<15}")
    print(f"  {'-'*45}")
    
    ab_mean_diff = df2[col].mean() - distributions_before[col].mean()
    ab_std_diff = df2[col].std() - distributions_before[col].std()
    print(f"  {'AdaBoost':<15} {ab_mean_diff:+.4f}        {ab_std_diff:+.4f}")
    
    for k in k_values:
        knn_mean_diff = knn_results[k][col].mean() - distributions_before[col].mean()
        knn_std_diff = knn_results[k][col].std() - distributions_before[col].std()
        print(f"  {'KNN(k='+str(k)+')':<15} {knn_mean_diff:+.4f}        {knn_std_diff:+.4f}")


# Merge des data

In [ ]:
# TODO

# Trouver un label de qualité

PCA en 2D sur les colonnes qui peuvent se rapporter à la qualité pour faire une visualisation dans le même style que celle du cours avec la criminalité pour espérer avoir un label cohérent. 

In [ ]:
# TODO

# Fit des modèles sur le label créé 

Tester différents modèles avec preprocessing adéquat...

In [ ]:
# TODO